In [62]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Yelp API Key
gkey = 'fxuoEjmZHd5kA1x20u4z2waSsXfTm49pWr2F6ac-htERJ0haCqXH949tlmDYt9lhy4MC6nBJjt2llMwhQJfxbwX69QlFkI221WkMDZTn7NMQBaXRrWGBFzCrg4-9XXYx'


In [63]:
from etl_clean_ip import zip_list

In [64]:
#set up base url
# base url
base_url = "https://api.yelp.com/v3/businesses/search"

# set up api_key dictionary
headers = {
        'Authorization': 'Bearer %s' % gkey,
    }

# set up a parameters dictionary
def s_params(target_loc,target_key,target_type):
    params = {
    "location": target_loc,
    "term": target_key,
    "categories": target_type,
}
    return params

# run a request using our params dictionary
def s_query(base_url,headers,params):
    response = requests.request('GET', base_url, headers=headers, params=params).json()
    return response

In [65]:
# Import zip list
zip_list 

#cities_pd.shape[0] #use .shape[0] to count the number of rows

[33142,
 33127,
 33147,
 33125,
 33136,
 33012,
 33167,
 33126,
 33150,
 33032,
 33130,
 33162,
 33034,
 33055,
 33056,
 33174,
 33172,
 33033,
 33168,
 33190,
 33169,
 33181,
 33030,
 33014,
 33193,
 33016,
 33161,
 33184,
 33194,
 33035]

In [149]:
# yelp function
def get_restaurants(loc):
    restaurants = []
    #these are the additional params
    add_params = {"limit" : 5, "sort_by": "best_match"}

    #these are the original params
    params = s_params(loc,"restaurants","restaurants, All")

    #add additional params to our params from before
    s1_params = {**params,**add_params}

    s1_results = s_query(base_url,headers,s1_params)
    
#     for result in s1_results:
    ix=[0,1,2,3,4,5]
    
    try:
        for i in ix:
       
            name = s1_results["businesses"][i]["name"]
            address1 = s1_results["businesses"][i]["location"]["address1"]
            city = s1_results["businesses"][i]["location"]["city"]
            state = s1_results["businesses"][i]["location"]["state"]
            zipcode = s1_results["businesses"][i]["location"]["zip_code"]
            rating = s1_results["businesses"][i]["rating"]
            price = s1_results["businesses"][i]["price"]
            restaurants.append([name,address1,city,zipcode,rating,price]) 
            
    except KeyError:
            print(f'Key not found. Missing data.')
            restaurants.append(["NaN","NaN","NaN","NaN","NaN","NaN"])
            
    return restaurants
  

In [150]:
get_restaurants(33145)

IndexError: list index out of range

In [140]:
# Loop through the zips list and run a search for each zip
names = []
streets = []
cities = []
zipcs = []
ratings = []
prices = []


print("wait...retrieving data")


for zips in zip_list:
    loc = zips
    output = get_restaurants(loc)
    names.append(output[0])
    streets.append(output[1])
    cities.append(output[2])
    zipcs.append(output[3])
    ratings.append(output[4])
    prices.append(output[5])
        
# # make exception for key errors        
#     except (KeyError, IndexError):
#         search = ["NaN","NaN","NaN","NaN","NaN","NaN"]
#     names.append(search[0])
#     streets.append(search[1])
#     cities.append(search[2])
#     zipcs.append(search[3])
#     ratings.append(search[4])
#     prices.append(search[5])
    
    
print("done!")

wait...retrieving data


KeyError: 'price'

In [ ]:
# Add columns for lat, lng, airport name, airport address, airport rating
restaurants_df = pd.DataFrame({"Name": names,"Location": streets, "City": cities, "Zipcode":zipcs, "Rating": ratings,"Price": prices})


In [69]:
restaurants_df.dropna(how="any", inplace=True)
restaurants_df

,Name,Location,City,Zipcode,Rating,Price
0,Hometown Bar-B-Que,1200 NW 22nd St,Miami,33142,4.5,$$
1,KYU,251 NW 25th St,Miami,33127,4.5,$$$
2,Sins Gastrobar,9835 NE 2nd Ave,Miami Shores,33138,4,$$
3,Old's Havana Cuban Bar & Cocina,1442 SW 8th St,Miami,33135,4.5,$$
4,Bunbury,55 NE 14th St,Miami,33132,4.5,$$
5,NaN,NaN,NaN,NaN,NaN,NaN
6,Sins Gastrobar,9835 NE 2nd Ave,Miami Shores,33138,4,$$
7,Amor di Pasta - Blue Lagoon,536 NW 57th Ave,Miami,33126,4.5,$$
8,Sins Gastrobar,9835 NE 2nd Ave,Miami Shores,33138,4,$$
9,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
# # Save Data to csv
# cities_pd.to_csv("Airport_Output.csv")

# # Visualize to confirm airport data appears
# cities_pd.head(10)